**On Local Machine**

In [2]:
!pip install transformers Pillow numpy tqdm

  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached tqdm-4.66.6-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.9.11-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached safetensors-0.4.5-cp311-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tokenizers-0.20.2-cp311-none-win_amd64.whl.metadata (6.9 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached charset_normalizer-3.4.0-cp311-cp311-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached ur

In [1]:
from PIL import Image
import torch
import os

In [2]:
import torch
import tensorflow as tf
print("PyTorch GPU available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("TensorFlow GPU available:", len(tf.config.list_physical_devices('GPU')) > 0)


PyTorch GPU available: True
Number of GPUs: 1
TensorFlow GPU available: False


Get the model

In [3]:
from transformers import AutoModelForImageClassification, AutoFeatureExtractor

model_name = "Organika/sdxl-detector"
processor = AutoFeatureExtractor.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForImageClassification.from_pretrained(model_name).to(device)


c:\Users\rahul\anaconda3\envs\deep_learning_hw\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [4]:
def predict_image_class(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt", padding=True).to(device)  # Move inputs to the same device
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    return model.config.id2label[predicted_class_idx]

In [5]:
real_folder = "test/REAL"
fake_folder = "test/FAKE"

REAL images accuracy

In [6]:
# Calculate accuracy for REAL images
real_correct = 0
real_total = 0
for filename in os.listdir(real_folder):
    image_path = os.path.join(real_folder, filename)
    prediction = predict_image_class(image_path)
    if prediction == "human":
        real_correct += 1
    real_total += 1
real_accuracy = (real_correct / real_total) * 100 if real_total > 0 else 0
print(f"REAL Images Detection Accuracy: {real_accuracy:.2f}%")

c:\Users\rahul\anaconda3\envs\deep_learning_hw\Lib\site-packages\transformers\image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ViTFeatureExtractor.preprocess` and were ignored: 'padding'
  return self.preprocess(images, **kwargs)


REAL Images Detection Accuracy: 85.77%


FAKE images accuracy

In [7]:
# Calculate accuracy for FAKE images
fake_correct = 0
fake_total = 0
for filename in os.listdir(fake_folder):
    image_path = os.path.join(fake_folder, filename)
    prediction = predict_image_class(image_path)
    if prediction == "artificial":
        fake_correct += 1
    fake_total += 1
fake_accuracy = (fake_correct / fake_total) * 100 if fake_total > 0 else 0
print(f"FAKE Images Detection Accuracy: {fake_accuracy:.2f}%")

FAKE Images Detection Accuracy: 29.89%
